In [83]:
import sys
import enum
import json
import pandas as pd
import gc


# needs a development version of python-tetris library, since
# TetrioEngine is not included in most recent release
sys.path.append('C:\\proj\\python-tetris')
import tetris

In [2]:
# %run ../tetris-bag.ipynb
# %run ../tetris-field.ipynb

In [4]:

with open('../tetrio-dl/replays/replay.json') as f:
# with open('../tetrio-dl/replays/test-replay-200-frames-garabage-travel.ttrm') as f:
    replay = json.load(f)
    
game = replay['data'][0]
inputs = game['replays'][0]['events']

# replay['data'][0]
# tetris.impl.presets.TetrioEngine()

In [131]:
# class InputKey(enum.IntEnum):
#     EMPTY = 0
#     moveLeft = enum.auto()
#     moveRight = enum.auto()
#     rotateCW = enum.auto()
#     rotateCCW = enum.auto()
#     hold = enum.auto()
#     hardDrop = enum.auto()
#     softDrop = enum.auto()
    
    
class KeyState(enum.Enum):
    down = False
    up = True


class KeyStateFrame():
    def __init__(self, state=KeyState.up, frame=-1):
        self.state = state
        self.frame = frame
    
    
class KeyStates():
    def __init__(self):
        self._states = {
            'moveLeft': KeyStateFrame(),
            'moveRight': KeyStateFrame(),
            'softDrop': KeyStateFrame()
        }
    
    def keydown(self, key, frame):
        print(f'[KEYDOWN] {key} at frame {frame}')
        if self._states[key].state == KeyState.down:
            raise Exception(f'key {key} was already pressed')
        
        self._states[key].state = KeyState.down
        self._states[key].frame = frame
    

    def keyup(self, key, frame):
        if self._states[key].state == KeyState.up:
            raise Exception(f'key {key} was not pressed')
        
        # time the key was pressed
        key_downtime = frame - self._states[key].frame
        
        print(f'[KEYUP] {key} at frame {frame}, downtime {key_downtime}')
        
        self._states[key].state = KeyState.up
        
        return key_downtime

In [138]:
class TetrioGame():
    def __init__(self, events_df, seed, options):
        self.events_df = events_df
        self.seed = seed
        self.options = options
        self.current_event_id = 3  # start with 4th event, since first three are only initialization
                
        self.sim_game = tetris.BaseGame(tetris.impl.presets.TetrioEngine, seed=seed)
        self.key_states = KeyStates()
        
    def __str__(self):
        return str(self.events_df)
    
    def _step(self):
        current_event = self.events_df.iloc[self.current_event_id]
        current_frame = current_event['frame']
        # current_time = current_frame * (50/3)
        
        if current_event['type'] == 'keydown' or current_event['type'] == 'keyup':
            current_key = current_event['key']
            current_subframe = current_event['data']['subframe']
            
            if current_key == 'moveLeft' or current_key == 'moveRight' or current_key == 'softDrop':
                # record keyup/keydown
                method = self.key_states.keydown if current_event['type'] == 'keydown' else self.key_states.keyup
                method(current_key, current_frame + current_subframe)
            else:
                # only execute it if key is pressed
                if current_event['type'] == 'keydown':
                    self._execute_action(current_key)
            
        elif current_event['type'] == 'ige':
            print('garbage')
            pass  # garbage
        
        print(current_event)
        
        self.current_event_id += 1
    
    def _execute_action(self, key):
        match key:
            case 'moveLeft':
                self.sim_game.left()
            case 'moveRight':
                self.sim_game.right()
            case 'rotateCW':
                self.sim_game.rotate(1)
            case 'rotateCCW':
                self.sim_game.rotate(-1)
            case 'hold':
                self.sim_game.swap()
            case 'hardDrop':
                self.sim_game.hard_drop()
            case 'softDrop':
                self.sim_game.soft_drop()
    
        
        
        
class TetrioReplay():
    def __init__(self, file_path):
        self.file_path = file_path
        
        with open(file_path) as f:
            replay = json.load(f)
            
        self.games = []
        
        # for each game in the replay
        for game in replay['data']:
            # it's enough to get one seed, since both player share the seed among one game
            seed = self._get_seed(game)
            
            # for each player in this game
            for player in game['replays']:
                # load dataframe
                event_df = pd.DataFrame(player['events'])
                # make key its own column
                event_df['key'] = events_df['data'].apply(lambda d: d['key'] if 'key' in d else None)
                
                # extract player options (arr [auto repeat rate], das [delayed auto shift], sdf [soft drop factor]
                options = self._get_options(event_df)
            
                # filter out all other events beside keydown
                # event_df = event_df.loc[event_df['type'] == 'keydown']
                # event_df['data'] = event_df['data'].apply(lambda d: d['key'])
                
                tg = TetrioGame(event_df, seed, options)
                self.games.append(tg)
                
            break
          
        
        # print(len(games))
        # force garbage collection
        del replay
        # gc.collect()
        
    def _get_seed(self, game_data):
        return game_data['replays'][0]['events'][1]['data']['options']['seed']
    
    def _get_options(self, event_data):
        # ARR (Auto Repeat Rate): the speed at which tetrominoes move when holding down movement keys, measured in frames per movement
        # DAS (Delayed Auto Shift): the time between the initial keypress and the start of its automatic repeat movement, measured in frames
        # DCD (DAS Cut Delay): if not 0, any ongoing DAS movement will pause for a set amount of time after dropping/rotating a piece, measured in frames
        # SDF (Soft Drop Factor): the factor with which soft drops change the gravity speed
        return event_data.iloc[1]['data']['options']['handling']
        
        


In [139]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output


r = TetrioReplay('../tetrio-dl/replays/replay.json')

for i in range(20):
    r.games[0]._step()

# def clicked(arg):
#     # print("button has been clicked!")
#     clear_output()
#     r.games[0]._step()
#     print(r.games[0].sim_game)

    
# button_download = widgets.Button(description = 'Test Button')
# button_download.on_click(clicked)
# display(button_download)

frame                                   1
type                              keydown
data     {'key': 'hold', 'subframe': 0.3}
key                                  hold
Name: 3, dtype: object
[KEYDOWN] softDrop at frame 3.9
frame                                       3
type                                  keydown
data     {'key': 'softDrop', 'subframe': 0.9}
key                                  softDrop
Name: 4, dtype: object
[KEYDOWN] moveRight at frame 4.5
frame                                        4
type                                   keydown
data     {'key': 'moveRight', 'subframe': 0.5}
key                                  moveRight
Name: 5, dtype: object
frame                                 6
type                              keyup
data     {'key': 'hold', 'subframe': 0}
key                                hold
Name: 6, dtype: object
[KEYUP] moveRight at frame 12.4, downtime 7.9
frame                                       12
type                                     keyup
dat

In [136]:
events_df = pd.DataFrame(replay['data'][0]['replays'][0]['events'])
# for d in events_df.loc[events_df['type'] == 'ige'].iterrows():
#     print(d[1]['data'])

# print('===================================================================\n' * 3)
    
# events_df = pd.DataFrame(replay['data'][0]['replays'][1]['events'])
# for d in events_df.loc[events_df['type'] == 'ige'].iterrows():
#     print(d[1]['data'])



In [126]:
events_df = pd.DataFrame(replay['data'][0]['replays'][0]['events'])
events_df['key'] = events_df['data'].apply(lambda d: d['key'] if 'key' in d else None)

# events_df['data'] = events_df['data'].apply(tuple)

events_df = events_df.loc[(events_df['type'] != 'keyup') | ((events_df['key'] != 'hardDrop') & (events_df['key'] != 'hold') & (events_df['key'] != 'rotateCCW') & (events_df['key'] != 'rotateCW'))]
# events_df = events_df.loc[events_df['type'] == 'keyup']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(events_df)

# type(events_df.iloc[3]['data']['key'])



     frame     type                                               data  \
0        0    start                                                 []   
1        0     full  {'successful': False, 'gameoverreason': None, ...   
2        0  targets  {'id': 'diyusi', 'frame': 0, 'type': 'targets'...   
3        1  keydown                   {'key': 'hold', 'subframe': 0.3}   
4        3  keydown               {'key': 'softDrop', 'subframe': 0.9}   
5        4  keydown              {'key': 'moveRight', 'subframe': 0.5}   
7       12    keyup              {'key': 'moveRight', 'subframe': 0.4}   
8       12    keyup               {'key': 'softDrop', 'subframe': 0.6}   
9       14  keydown               {'key': 'hardDrop', 'subframe': 0.4}   
11      20  keydown                 {'key': 'hardDrop', 'subframe': 0}   
13      24  keydown               {'key': 'moveLeft', 'subframe': 0.1}   
14      31    keyup                 {'key': 'moveLeft', 'subframe': 0}   
15      32  keydown               {'ke

In [ ]:
'